In [1]:
import time, datetime
from fyers_apiv3 import fyersModel
client_id = "ZHQ4IJL7TI-100"
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC
from indicators.candle import OHLC

In [4]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_today, date_100_p

('2024-11-23', '2024-08-15')

In [5]:
response = fyers.history(data={"symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                 "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

len(response['candles']), response

(25185,
 {'candles': [[1723779900, 134.75, 136.28, 134.5, 135.26, 218188],
   [1723779960, 135.26, 135.6, 135.26, 135.53, 40267],
   [1723780020, 135.53, 135.53, 135.15, 135.3, 44082],
   [1723780080, 135.3, 135.45, 135.14, 135.3, 29135],
   [1723780140, 135.45, 135.45, 135, 135.15, 69106],
   [1723780200, 135.13, 135.25, 134.88, 135.01, 47452],
   [1723780260, 135.01, 135.39, 135.01, 135.2, 51405],
   [1723780320, 135.2, 135.35, 135.05, 135.3, 106606],
   [1723780380, 135.34, 135.34, 135.05, 135.27, 24634],
   [1723780440, 135.27, 135.3, 135.1, 135.12, 27130],
   [1723780500, 135.12, 135.17, 134.9, 135.14, 32472],
   [1723780560, 135.14, 135.15, 134.92, 134.95, 9265],
   [1723780620, 134.8, 134.99, 134.76, 134.99, 22534],
   [1723780680, 134.99, 135, 134.8, 134.93, 15832],
   [1723780740, 134.93, 135.07, 134.86, 135.07, 9956],
   [1723780800, 135.07, 135.15, 134.94, 135.06, 33027],
   [1723780860, 134.95, 135.05, 134.85, 135.04, 27262],
   [1723780920, 135.04, 135.16, 134.92, 135.05, 

In [6]:
candles = np.array(response['candles'])
type(candles), candles

(numpy.ndarray,
 array([[1.72377990e+09, 1.34750000e+02, 1.36280000e+02, 1.34500000e+02,
         1.35260000e+02, 2.18188000e+05],
        [1.72377996e+09, 1.35260000e+02, 1.35600000e+02, 1.35260000e+02,
         1.35530000e+02, 4.02670000e+04],
        [1.72378002e+09, 1.35530000e+02, 1.35530000e+02, 1.35150000e+02,
         1.35300000e+02, 4.40820000e+04],
        ...,
        [1.73226942e+09, 1.16990000e+02, 1.16990000e+02, 1.16960000e+02,
         1.16990000e+02, 2.96590000e+04],
        [1.73226948e+09, 1.16990000e+02, 1.16990000e+02, 1.16910000e+02,
         1.16980000e+02, 6.26250000e+04],
        [1.73226954e+09, 1.16990000e+02, 1.17110000e+02, 1.16920000e+02,
         1.16950000e+02, 1.06265000e+05]]))

In [7]:
sync = SyncInd( 
    SMA(5),
    SMA(15),
    Alligator(),
    # RSI(),
    # ATR(),
    VolumeROC(),
                
)
                
for c in candles:
    sync.append(c)

In [8]:
X, y = [], []

last_candles_count = 8
next_candles_count = 1

temp = []
for ind, i in enumerate(sync.data()):
    k = i.tolist()
    temp.append(k[1:5] + k[6:]) # deleting date and volume from the data
current_data = np.array(temp, dtype=np.float32)
# current_data = sync.data()[:, 1:]

for ind in range(50+(last_candles_count), current_data.shape[0]-50-(next_candles_count-1)):

    first_candle = current_data[ind-last_candles_count]
    temp = []
    for i in range(1, last_candles_count):
        temp.append(first_candle - current_data[(ind-last_candles_count)+i])

    X.append(temp)
    temp = []
    for j in range(next_candles_count):
        temp.append((first_candle - current_data[(ind + j)])[1])
    y.append(temp)
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [9]:
X.shape, y.shape

((25077, 7, 10), (25077, 1))

In [10]:
import torch
import torch.nn as nn
import numpy as np

In [11]:
# Transformer Model for Market Data
class MarketTransformer(nn.Module):
    def __init__(self, input_dim, seq_len, d_model, num_heads, num_layers, ff_dim, dropout=0.1, output_dim=1):
        super(MarketTransformer, self).__init__()
        
        # Embedding Layers
        self.feature_embedding = nn.Linear(input_dim, d_model)
        self.positional_encoding = self._generate_positional_encoding(seq_len, d_model)
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=num_heads, 
            dim_feedforward=ff_dim, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output Layer
        self.output_layer = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, output_dim)  # Modify output_dim based on your task
        )
    
    def _generate_positional_encoding(self, seq_len, d_model):
        # Generate positional encodings for temporal sequences
        position = torch.arange(seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pos_encoding = torch.zeros(seq_len, d_model)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding.unsqueeze(0)  # Add batch dimension
    
    def forward(self, x):
        # Embed features
        x = self.feature_embedding(x)  # (batch_size, seq_len, d_model)
        x = x + self.positional_encoding.to(x.device)  # Add positional encoding
        
        # Transformer Encoder
        x = self.transformer_encoder(x)  # (batch_size, seq_len, d_model)
        
        # Pooling and Output
        x = x.mean(dim=1)  # Global average pooling over sequence length
        output = self.output_layer(x)
        return output

# Hyperparameters
# input_dim = 12  # Features per timestep
# seq_len = 7  # Timesteps
# d_model = 64  # Embedding dimension
# num_heads = 4  # Number of attention heads
# num_layers = 3  # Transformer layers
# ff_dim = 128  # Feedforward layer dimension
# dropout = 0.1
# output_dim = 1  # Single output for prediction (e.g., future trend)

# Instantiate the model
# model = MarketTransformer(
#     input_dim=input_dim,
#     seq_len=seq_len,
#     d_model=d_model,
#     num_heads=num_heads,
#     num_layers=num_layers,
#     ff_dim=ff_dim,
#     dropout=dropout,
#     output_dim=output_dim
# )

# # Example Input
# batch_size = 32
# dummy_input = torch.randn(batch_size, seq_len, input_dim)  # (batch_size, seq_len, input_dim)

# # Forward Pass
# output = model(dummy_input)
# print("Output shape:", output.shape)  # Should be (batch_size, output_dim)

In [12]:
import torch
import numpy as np

# Assuming data is a numpy array of shape (1000, 7, 12)
data = X[:, :, 1:]  # Replace with your actual data

# Convert to PyTorch tensor
data_tensor = torch.tensor(data, dtype=torch.float32)

In [13]:
# Split features and labels
features = torch.tensor(X[:, :, 1:], dtype=torch.float32)  # Use all but last timestep for features
labels = torch.tensor(y, dtype=torch.float32)    # Use first feature of last timestep as label

# Check shapes
print("Features shape:", features.shape)  # (1000, 6, 12)
print("Labels shape:", labels.shape)      # (1000,)

Features shape: torch.Size([25077, 7, 9])
Labels shape: torch.Size([25077, 1])


In [14]:
from torch.utils.data import Dataset, DataLoader

class MarketDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create dataset
dataset = MarketDataset(features, labels)

# Split into train and test sets
train_size = int(0.8 * len(dataset))  # 80% train, 20% test
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
model = MarketTransformer(
    input_dim=11,    # Number of features
    seq_len=7,       # Number of timesteps
    d_model=32,      # Embedding dimension
    num_heads=4,     # Number of attention heads
    num_layers=5,    # Transformer layers
    ff_dim=128,      # Feedforward dimension
    dropout=0.1,     # Dropout rate
    output_dim=1     # Predict a single value
)

In [16]:
import torch.nn as nn
import torch.optim as optim

# Loss function: Mean Squared Error for regression
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Learning rate scheduler (optional)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

In [17]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch_features, batch_labels in train_loader:
        optimizer.zero_grad()  # Clear gradients

        # Forward pass
        predictions = model(batch_features)
        predictions = predictions.squeeze()  # Adjust shape if needed

        # Compute loss
        loss = criterion(predictions, batch_labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Adjust learning rate
    scheduler.step()

    # Log training loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (224x9 and 11x32)

In [ ]:
model.eval()  # Set model to evaluation mode
total_loss = 0

with torch.no_grad():  # Disable gradient computation
    for batch_features, batch_labels in test_loader:
        predictions = model(batch_features)
        predictions = predictions.squeeze()

        # Compute loss
        loss = criterion(predictions, batch_labels)
        total_loss += loss.item()

print(f"Test Loss: {total_loss / len(test_loader):.4f}")

In [ ]:
# Example: Predict on new samples
new_data = X[-10:, :, 1:]  # Replace with actual new data
new_data_tensor = torch.tensor(new_data, dtype=torch.float32)

# Get predictions
model.eval()
with torch.no_grad():
    predictions = model(new_data_tensor)
    print("Predictions:\n", predictions.numpy(), "\n\n", y[-10:])


In [ ]:
import matplotlib.pyplot as plt

# Plot all predictions vs. actual values
plt.figure(figsize=(12, 6))
plt.plot(y[-10:], label="Actual Values", color="blue", alpha=0.7)
plt.plot(predictions.numpy(), label="Predicted Values", color="red", alpha=0.7)
plt.title("Predictions vs Actual Values")
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.legend()
plt.show()